In [2]:
import pandas as pd
import re

from langchain_anthropic import ChatAnthropic
from langchain.embeddings import HuggingFaceEmbeddings

from agents.vectorstore import Vectorstore
from agents.qa_generation.qa_generation import QAGeneration
from agents.qa_eval_context.qa_eval_context import QAContextEval

from dotenv import load_dotenv
load_dotenv()

True

In [3]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)
vectorstore = Vectorstore(embeddings=embeddings, save=False)
pdf_docs = ['./Documents/Analisis-COP3-Escazu_ONG-CEUS-Chile.pdf']
store_name = 'test_vectorstore'
vecs = vectorstore.generate_vectorstore(pdf_docs, store_name, chunk_size=2500, chunk_overlap=300)

/home/joseterr/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/joseterr/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/joseterr/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4

In [4]:
llm = ChatAnthropic(model='claude-3-haiku-20240307', temperature=0.3)
qa_chain = QAGeneration(llm=llm)

In [5]:
input_text = 'acciones por el cambio'
chunks_similarities = vecs.similarity_search(input_text, k=10)
qa_generated = []
for chunk in chunks_similarities:
    resp = qa_chain.chain.invoke(chunk.page_content)
    text = resp['text']
    question = resp['questions']['question']
    answer = resp['questions']['answer']
    qa_generated.append({'text': text, 'question': question, 'answer': answer})

In [7]:
qa_generated[0]

{'text': 'dares introducidos por Escazú, pese a que no todo país haya firmado y ratificado el\nacuerdo. La CIDH debe interpretar derechos de acceso de manera evolutiva dentro\nde la convención interamericana, por lo cual la sociedad civil ha buscado poder in-\ntegrar las ideas desarrolladas en el Acuerdo de Escazú. Aquello, al argumentar que\nel acuerdo es evidencia de un creciente consenso respecto a derechos de acceso en\nmateria medioambiental. Durante el mes de abril y mayo la sociedad civil empujó al\nreconocimiento de estas materias a través de la función consultiva de la CIDH para\nintegrar la opinión pública. Se realizó una submission a través de un amicus sobre la\nrelación de estado con personas, organizaciones, y grupos defensores del ambiente.\nLa pre-COP fue un lugar en el que se discutió mucho de cómo debe haber un ánimo\nde no contravenir el espíritu del acuerdo por los Estados Parte, en especial en lo\nrelacionado con las obligaciones establecidas en el artículo 9. Se e

In [8]:
llm = ChatAnthropic(model='claude-3-haiku-20240307', temperature=0.3)
eval_chain = QAContextEval(llm=llm)
i = 0


context = qa_generated[i]['text']
query = qa_generated[i]['question']
result = 'Comprar mineras'
resp = eval_chain.evaluate(query, context, result)
resp